In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import random

import accelerate
import torch
import transformers

from src._shared import (
    apply_lora_to_model,
    apply_peft_to_model,
    freeze_base_models,
    load_clip_model,
    load_config,
    load_tokenizers,
    prepare_dataset,
    save_model_and_logs,
    setup_environment,
    setup_trainer,
    train_model,
)

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

train_config = load_config()
model_name_identifier, device, report_to, run, USE_WANDB, SEED = setup_environment(train_config)

accelerate.utils.set_seed(SEED + 1)
transformers.set_seed(SEED + 2)
torch.manual_seed(SEED + 3)
random.seed(SEED + 4)

wandb: Currently logged in as: finnlueth (finnlueth-organization). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Using device: cuda
Model identifier: protT5-CLIP-2025-01-12-19-58-05


In [4]:
tokenizer_plm, tokenizer_llm = load_tokenizers(train_config)
dataset = prepare_dataset(train_config, tokenizer_plm, tokenizer_llm)

Loading dataset from disk...


In [5]:
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['identifier', 'term', 'aspect', 'GO Name', 'GO Sentence', 'sequence', 'species', '__index_level_0__', 'sequence_processed', 'input_ids_sequence', 'attention_mask_sequence', 'input_ids_text', 'attention_mask_text'],
        num_rows: 44098
    })
    test: Dataset({
        features: ['identifier', 'term', 'aspect', 'GO Name', 'GO Sentence', 'sequence', 'species', '__index_level_0__', 'sequence_processed', 'input_ids_sequence', 'attention_mask_sequence', 'input_ids_text', 'attention_mask_text'],
        num_rows: 221346
    })
})
{'identifier': 'A0A023GUT0', 'term': 'GO:0042531', 'aspect': 'BPO', 'GO Name': 'positive regulation of tyrosine phosphorylation of STAT protein', 'GO Sentence': 'The biological process is positive regulation of tyrosine phosphorylation of STAT protein.', 'sequence': 'MRCPGVSLWGLLCLGAAAGGGRPVRLEGLRADARALTRTLSTRLQQLQLFPLTLRLSGLEGVPEGVPEGVPEGGVPPGLGWAAQRLQLFQRLLGALPGPDPRLAQVANDLENLRSLLALLGTLLGCPPPRDPRPPPPAPLAEA

In [6]:
model = load_clip_model(train_config, device)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded model...
All model parameters are on CUDA


In [7]:
train_config['lora']['enabled']

True

In [8]:
if train_config['lora']['enabled']:
    model = apply_lora_to_model(model, train_config)
else:
    model = apply_peft_to_model(model, train_config)

target_modules: ['q', 'v']
modules_to_save: ['protein_projection', 'text_projection', 'logit_scale']
trainable params: 6,160,385 || all params: 5,039,576,066 || trainable%: 0.1222


In [9]:
trainer = setup_trainer(model, dataset, train_config, model_name_identifier, USE_WANDB, tokenizer_plm, tokenizer_llm)

In [10]:
train_model(trainer)

You are not running the flash-attention implementation, expect numerical differences.


/home/finnlueth/repos/prostT5-CLIP/.venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in  - will assume that the vocabulary was not modified.
  warnings.warn(


In [12]:
save_model_and_logs(model, trainer, model_name_identifier, train_config)

/home/finnlueth/repos/prostT5-CLIP/.venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in  - will assume that the vocabulary was not modified.
  warnings.warn(


Model, config, and log saved to: ../tmp/models/protT5-CLIP-2025-01-12-19-58-05


In [11]:
# if train_config.lora.enabled:
#     model = apply_lora_to_model(model, train_config)
# else:
#     freeze_base_models(model)